In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# IMPORTING LIBRARIES
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import warnings
warnings.filterwarnings("ignore")
from wordcloud import WordCloud,STOPWORDS
import io
import base64
from matplotlib import rc,animation
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import os
print(os.listdir("../input"))

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

##  READING TRAIN AND TEST DATA

In [ ]:
data1=pd.read_csv("/kaggle/input/home-credit-default-risk/application_train.csv")
df_train=data1.copy()
df_train.tail()

In [ ]:
data2=pd.read_csv("/kaggle/input/home-credit-default-risk/application_test.csv")
df_test=data2.copy()
df_test.tail()

In [ ]:
len(df_test.columns)

In [ ]:
len(df_train.columns)

### WE CONCAT TRAIN AND TEST

In [ ]:
df_train['data_type'] = 'train'
df_test['data_type'] = 'test'
df = pd.concat([df_train,df_test])

In [ ]:
df.tail()

* TARGET value 0 means loan is repayed, value 1 means loan is not repayed.

### TARGET DISTRIBUTION

In [ ]:
df["TARGET"].value_counts()

In [ ]:
df["TARGET"].isnull().sum()  #test datasinin Nan degerleri

In [ ]:
df_train['TARGET'].astype(int).plot.hist();

* The target variable is slightly imbalance with the majority of loans has the target equals to 0 which indicates that individuals did not had any problems in paying installments in given time.

## Percentage of Missing values in application train and test data

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [ ]:
missing_data(df_train).head(20)

In [ ]:
missing_data(df_test).head(20)

In [ ]:
fig = plt.figure(figsize=(18,6))
miss_train = pd.DataFrame((df_train.isnull().sum())*100/df_train.shape[0]).reset_index()
miss_test = pd.DataFrame((df_test.isnull().sum())*100/df_test.shape[0]).reset_index()
miss_train["type"] = "train"
miss_test["type"]  =  "test"
missing = pd.concat([miss_train,miss_test],axis=0)
ax = sns.pointplot("index",0,data=missing,hue="type")
plt.xticks(rotation =90,fontsize =7)
plt.title("Percentage of Missing values in application train and test data")
plt.ylabel("PERCENTAGE")
plt.xlabel("COLUMNS")
ax.set_facecolor("k")
fig.set_facecolor("lightgrey")

* CONCLUSION: Missing values of train and test almost same.

In [ ]:
import seaborn as sns
sns.heatmap(df.isnull(), cbar = False);

In [ ]:
import missingno as msno
msno.matrix(df.sample(500), inline=True, sparkline=True, figsize=(20,10), sort=None, color=(0.25, 0.45, 0.6))

* When they zoom in on the central pillars, we see that they mostly get information about the building where the customer lives. It appears that there are many applicants who leave the information blank for their residence. We can think of why this might be so or how it could inform our model.

* This time I will sort the data to see the odds better.This time I will sort the data to see the odds better.

In [ ]:
msno.matrix(df.iloc[0:100, 40:94], inline=True, sparkline=True, figsize=(20,10), sort='ascending', fontsize=12, labels=True, color=(0.25, 0.45, 0.6))

In [ ]:
#We can see amount of missing values in bar grafic
import missingno as msno
msno.bar(df);

In [ ]:
msno.heatmap(df); 

In [ ]:
#We can see relation between missing values in dendrogram
msno.dendrogram(df)
plt.show()

## EXPLATORY DATA

In [ ]:
df.dtypes.value_counts()

In [ ]:
# Number of unique classes in each object column 
#WE USE LEBEL ENCODER FOR 2 VALUES AND ONE-HOT FOR MORE THAN 2.We will use it later
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

### LETS LOOK AT CORRELATION BETWEEN TARGET AND ALL VARIABLES

In [ ]:
cor = df.corr()['TARGET'].sort_values()
print('Most Positive Correlations:\n', cor.tail(15))
print('\nMost Negative Correlations:\n', cor.head(15))

* Correlations between target and other features are not strong.

In [ ]:
df.columns.values

* There are total 307,511 rows which contains the information of loans and there are 122 variables.

* The target variable defines if the client had payment difficulties meaning he/she had late payment more than X days on at least one of the first Y installments of the loan. Such case is marked as 1 while other all other cases as 0.

## LOOK SUMMARY OUITLIER FROM DESCRIBE STATISTICS

In [ ]:
df.describe().T

In [ ]:
df.head()

## MANUPULATION OF FEATURES

### 1 ) lets combine 4 features(23,24,25,26) 
'FLAG_MOBIL', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL'

In [ ]:
df['PHONE'] = df[['FLAG_MOBIL','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL']].sum(axis = 1)

In [ ]:
df.drop(df[['FLAG_MOBIL','FLAG_CONT_MOBILE','FLAG_PHONE','FLAG_EMAIL']],axis=1,inplace=True)

### 2 ) lets combine  6 features(35,36,37,38,39,40)
#### Mismatched information...
REG_REGION_NOT_LIVE_REGION, REG_REGION_NOT_WORK_REGION,  LIVE_REGION_NOT_WORK_REGION,  REG_CITY_NOT_LIVE_CITY,  REG_CITY_NOT_WORK_CITY, LIVE_CITY_NOT_WORK_CITY
* We can sum mismatched information and create a new column WRONG_ADRESS 

In [ ]:
df['WRONG_ADRESS'] = df[['REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY']].sum(axis = 1)

In [ ]:
df.drop(df[['REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY']],axis=1,inplace=True)

In [ ]:
df['WRONG_ADRESS'].corr(df['TARGET'])

### 3 ) FLAG DOCUMENTS 2-21

In [ ]:
print(df['FLAG_DOCUMENT_2'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_3'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_4'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_5'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_6'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_7'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_8'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_8'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_9'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_10'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_11'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_12'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_13'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_14'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_15'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_16'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_17'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_18'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_19'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_20'].corr(df['TARGET']))
print(df['FLAG_DOCUMENT_21'].corr(df['TARGET']))

* Document 2,3,20,21 is positive correlation and others are negative correlations.
* Thus we decided to create 2 different columns DOCUMENT_SUM_1 for negatif correletion and DOCUMENT_SUM_2 for positive correlation. 

In [ ]:
df['DOCUMENT_SUM_1'] = (df.loc[:,'FLAG_DOCUMENT_4':'FLAG_DOCUMENT_19'].sum(axis = 1))/(16)
df['DOCUMENT_SUM_1'].corr(df['TARGET'])

In [ ]:
df['DOCUMENT_SUM_2'] = (df[['FLAG_DOCUMENT_2','FLAG_DOCUMENT_3','FLAG_DOCUMENT_20','FLAG_DOCUMENT_21']].sum(axis = 1))/(4)
df['DOCUMENT_SUM_2'].corr(df['TARGET'])

In [ ]:
df.drop(df.loc[:,'FLAG_DOCUMENT_2':'FLAG_DOCUMENT_21'],axis=1,inplace=True)

In [ ]:
df.columns

In [ ]:
df['DOCUMENT_SUM_1'].sample(20)

### 5 ) INFORMATION FEATURES ABOUT BUILDING WHERE THE CLIENT LIVES(45,46,...91)
* There are numerical and  4 categorical variables.We will examine it separately. 

In [ ]:
df_train.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE']

In [ ]:
crr=df_train.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE'].corr() 

In [ ]:
df_train.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE'].corr()

In [ ]:
# Heatmap of correlations
plt.figure(figsize = (28, 26))
sns.heatmap(crr, cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

In [ ]:
#print(df_train['EMERGENCYSTATE_MODE'].corr(df['TARGET'])#kate
#print(df_train['WALLSMATERIAL_MODE'].corr(df['TARGET']))#kate          
print(df_train['TOTALAREA_MODE'].corr(df['TARGET']))
#print(df_train['HOUSETYPE_MODE'].corr(df['TARGET']))#kategorik          
#print(df_train['FONDKAPREMONT_MODE'].corr(df['TARGET']))#kategorik
print("------------------------------------------------------------------")
print("1",df_train['NONLIVINGAREA_MEDI'].corr(df['TARGET']))          
print("2",df_train['NONLIVINGAPARTMENTS_MEDI'].corr(df['TARGET']))
print("3",df_train['LIVINGAREA_MEDI'].corr(df['TARGET']))
print("4",df_train['LIVINGAPARTMENTS_MEDI'].corr(df['TARGET']))
print("5",df_train['LANDAREA_MEDI'].corr(df['TARGET']))
print("6",df_train['FLOORSMIN_MEDI'].corr(df['TARGET']))
print("7",df_train['FLOORSMAX_MEDI'].corr(df['TARGET']))
print("8",df_train['ENTRANCES_MEDI'].corr(df['TARGET']))
print("9",df_train['ELEVATORS_MEDI'].corr(df['TARGET']))
print("10",df_train['COMMONAREA_MEDI'].corr(df['TARGET']))
print("11",df_train['YEARS_BUILD_MEDI'].corr(df['TARGET']))
print("12",df_train['YEARS_BEGINEXPLUATATION_MEDI'].corr(df['TARGET']))
print("13",df_train['BASEMENTAREA_MEDI'].corr(df['TARGET']))
print("14",df_train['APARTMENTS_MEDI'].corr(df['TARGET']))
print("---------------------------------------------------------")
print("1",df_train['NONLIVINGAREA_MODE'].corr(df['TARGET']))
print("2",df_train['NONLIVINGAPARTMENTS_MODE'].corr(df['TARGET']))
print("3",df_train['LIVINGAREA_MODE'].corr(df['TARGET']))
print("4",df_train['LIVINGAPARTMENTS_MODE'].corr(df['TARGET']))
print("5",df_train['LANDAREA_MODE'].corr(df['TARGET']))
print("6",df_train['FLOORSMIN_MODE'].corr(df['TARGET']))
print("7",df_train['FLOORSMAX_MODE'].corr(df['TARGET']))
print("8",df_train['ENTRANCES_MODE'].corr(df['TARGET']))
print("9",df_train['ELEVATORS_MODE'].corr(df['TARGET']))
print("10",df_train['COMMONAREA_MODE'].corr(df['TARGET']))
print("11",df_train['YEARS_BUILD_MODE'].corr(df['TARGET']))
print("12",df_train['YEARS_BEGINEXPLUATATION_MODE'].corr(df['TARGET']))
print("13",df_train['BASEMENTAREA_MODE'].corr(df['TARGET']))
print("14",df_train['APARTMENTS_MODE'].corr(df['TARGET']))
print("-------------------------------------------------------------------")
print("1",df_train['NONLIVINGAREA_AVG'].corr(df['TARGET']))
print("2",df_train['NONLIVINGAPARTMENTS_AVG'].corr(df['TARGET']))
print("3",df_train['LIVINGAREA_AVG'].corr(df['TARGET']))
print("4",df_train['LIVINGAPARTMENTS_AVG'].corr(df['TARGET']))
print("5",df_train['LANDAREA_AVG'].corr(df['TARGET']))
print("6",df_train['FLOORSMIN_AVG'].corr(df['TARGET']))
print("7",df_train['FLOORSMAX_AVG'].corr(df['TARGET']))
print("8",df_train['ENTRANCES_AVG'].corr(df['TARGET']))
print("9",df_train['ELEVATORS_AVG'].corr(df['TARGET']))
print("10",df_train['COMMONAREA_AVG'].corr(df['TARGET']))
print("11",df_train['YEARS_BUILD_AVG'].corr(df['TARGET']))
print("12",df_train['YEARS_BEGINEXPLUATATION_AVG'].corr(df['TARGET']))
print("13",df_train['BASEMENTAREA_AVG'].corr(df['TARGET']))
print("14",df_train['APARTMENTS_AVG'].corr(df['TARGET']))


In [ ]:
df.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE'].dtypes.value_counts()

In [ ]:
df.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE'].columns  # columns number 45 to 91

In [ ]:
df.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE'].sample(10)

### 5/a ) LETS LOOK AT CATEGORICAL VARIABLES OF BUILDINGS FEATURES 

In [ ]:
#1
df["EMERGENCYSTATE_MODE"].unique()

In [ ]:
df["EMERGENCYSTATE_MODE"].value_counts()

In [ ]:
df["EMERGENCYSTATE_MODE"].isnull().sum()  #bu nan value ler unknown yapilabilir mi?/

In [ ]:
#2
df["WALLSMATERIAL_MODE"].unique()   
#nan lar mod medyan mantiksiz cok nan value var,one-hot yapildiktan sonra makine ile doldurulabilir

In [ ]:
df["WALLSMATERIAL_MODE"].value_counts()

In [ ]:
df["WALLSMATERIAL_MODE"].isnull().sum()

In [ ]:
#3
df["FONDKAPREMONT_MODE"].unique()

In [ ]:
df["FONDKAPREMONT_MODE"].value_counts()  #nan value cok.One-hot dan sonra makine ile doldurmak mantikli.

In [ ]:
df["FONDKAPREMONT_MODE"].isnull().sum()

In [ ]:
#4
df["HOUSETYPE_MODE"].unique()

In [ ]:
df["HOUSETYPE_MODE"].value_counts()#mod ile doldurulabilir veya makine ile

In [ ]:
df["HOUSETYPE_MODE"].isnull().sum()

### 5/b ) LETS LOOK AT NUMERICAL VARIABLES OF BUILDINGS FEATURES

In [ ]:
df.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE'].dtypes.value_counts()

In [ ]:
df.loc[:,'APARTMENTS_AVG':'EMERGENCYSTATE_MODE'].dtypes

In [ ]:
df['TARGET'].corr(df.loc[:,'APARTMENTS_AVG':'NONLIVINGAREA_MEDI'].sum(axis = 1))

* we decided to use median for missing values.

In [ ]:
for col in df.loc[:,'APARTMENTS_AVG':'NONLIVINGAREA_MEDI'].columns:
    df[col] = df[col].fillna(df[col].median())

In [ ]:
df['TARGET'].corr((df.loc[:,'APARTMENTS_AVG':'NONLIVINGAREA_MEDI'].sum(axis = 1))*(3-df['REGION_RATING_CLIENT_W_CITY']))

* If we use variable 'REGION_RATING_CLIENT_W_CITY',correlation increases.
* We decided to sum all variables and create a new variable 'BUILDING_FEATURES'

In [ ]:
# BUILDING_FEATURES değişkeni veriye ekleniyor
df['BUILDING_FEATURES'] = (df.loc[:,'APARTMENTS_AVG':'NONLIVINGAREA_AVG'].sum(axis = 1))*(3-df['REGION_RATING_CLIENT_W_CITY'])

In [ ]:
df.drop(df.loc[:,'APARTMENTS_AVG':'NONLIVINGAREA_MEDI'],axis=1,inplace=True)

In [ ]:
list(df.columns)

### 6 ) CODE_GENDER

In [ ]:
df['CODE_GENDER'].value_counts()

In [ ]:
#df.drop(df[df['CODE_GENDER']=='XNA'].index, inplace=True)

* We decided to replace 4 XNA to F. Because F is more than M

In [ ]:
df['CODE_GENDER'].replace('XNA', 'F',inplace=True)  

In [ ]:
df['CODE_GENDER'].value_counts()

### 7 ) DAYS_BIRTH
* The age of the customer in days at the time of application. The number of days must be converted to years.

In [ ]:
df['DAYS_BIRTH'].sample(20)

In [ ]:
df['DAYS_BIRTH']=df['DAYS_BIRTH']/ -365

In [ ]:
df['DAYS_BIRTH'].sample(20)

## ANALYSIS OF OTHER CATEGORICALS VARIABLES

In [ ]:
df.head()

In [ ]:
df.iloc[:,37:].head()

In [ ]:
display(df['FLAG_OWN_CAR'].value_counts())
display(df['FLAG_OWN_REALTY'].value_counts())
display(df['NAME_TYPE_SUITE'].value_counts())
display(df['NAME_INCOME_TYPE'].value_counts())
display(df['NAME_EDUCATION_TYPE'].value_counts())
display(df['NAME_FAMILY_STATUS'].value_counts())
display(df['NAME_HOUSING_TYPE'].value_counts())
display(df['OCCUPATION_TYPE'].value_counts())
display(df['WEEKDAY_APPR_PROCESS_START'].value_counts())
display(df['ORGANIZATION_TYPE'].value_counts())
display(df['FONDKAPREMONT_MODE'].value_counts())
display(df['HOUSETYPE_MODE'].value_counts())
display(df['WALLSMATERIAL_MODE'].value_counts())
display(df['EMERGENCYSTATE_MODE'].value_counts())


### 8 ) DAYS_EMPLOYED
* This variable gives the number of days the loan applicant has worked.

In [ ]:
df['DAYS_EMPLOYED'].value_counts()

In [ ]:
df['DAYS_EMPLOYED'].describe()

* We make 365243 days for about 1000 years, a value that is impossible in reality, and we convert it to Nan value.
* These data are automatically filled by the system (64648 data), actually NaN data.

In [ ]:
df['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [ ]:
df['DAYS_EMPLOYED'].isnull().sum()

## LETS LOOK AT ALL FEATURES CORRELATIONS

In [ ]:
# Heatmap of correlations
plt.figure(figsize = (28, 26))
sns.heatmap(df.corr(), cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
plt.title('Correlation Heatmap');

* Correlation 'OBS_30_CNT_SOCIAL_CIRCLE' and 'OBS_60_CNT_SOCIAL_CIRCLE' is strong 100% and same correlation with target
* 'DEF_30_CNT_SOCIAL_CIRCLE' with 'DEF_60_CNT_SOCIAL_CIRCLE' correlation 90% and target and corelations are the same
* AMT_ANNUITY, AMT_GOODS_PRICE, AMT_CREDIT correlations are strong
* REGION_RATING_CLIENT, REGION_RATING_CLIENT_W_CITY correlation strong


### 9 ) AMT_REQ_CREDIT_BUREAU_HOUR/DAY/WEEK/MON/QRT/YEAR
* The number of inquiries about the customer one day before the application to the Credit Bureau (except one hour / day / month / 3 months / year)
* They have NaN values at the same time

In [ ]:
df.loc[:,'AMT_REQ_CREDIT_BUREAU_HOUR':'AMT_REQ_CREDIT_BUREAU_YEAR'].sample(15)

In [ ]:
df.loc[:,'AMT_REQ_CREDIT_BUREAU_HOUR':'AMT_REQ_CREDIT_BUREAU_YEAR'].isnull().sum()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_YEAR'].value_counts()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_QRT'].value_counts()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_MON'].value_counts()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_WEEK'].value_counts()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_DAY'].value_counts()

In [ ]:
df['AMT_REQ_CREDIT_BUREAU_HOUR'].value_counts()

* 6 feature da ayni anda nan value var muhtemelen soru olmadigini gosteriyor bunlari 0 ile doldurabiliriz...

In [ ]:
list(df.columns)

In [ ]:
msno.bar(df);

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
msno.matrix(df.sample(200));

In [ ]:
msno.matrix(df);

In [ ]:
msno.heatmap(df);

## SOME OF ANALYSIS OF MISSINGS VALUES

### 1 ) OWN_CAR_AGE ,FLAG_OWN_CAR

In [ ]:
df['OWN_CAR_AGE'].isnull().sum()

In [ ]:
#df['OWN_CAR_AGE'].value_counts()

In [ ]:
df['FLAG_OWN_CAR'].value_counts()

In [ ]:
df['OWN_CAR_AGE'].corr(df['TARGET'])

In [ ]:
df[df['OWN_CAR_AGE']==0.0].sample(5)

In [ ]:
df[['OWN_CAR_AGE','FLAG_OWN_CAR','TARGET']].sample(20)

* CONCLUSION : We understand from here that it is filled as Nan, which does not have a car.We can not write a numerical variables for car age.We will predict from machine learning methods.

### 2 ) 'OCCUPATION_TYPE','NAME_INCOME_TYPE' 

In [ ]:
df[['OCCUPATION_TYPE','NAME_INCOME_TYPE','TARGET']].sample(20)

### 3 ) 'AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY', 'AMT_GOODS_PRICE',"TARGET" correlation:

In [ ]:
amt = df[[ 'AMT_INCOME_TOTAL','AMT_CREDIT',
                         'AMT_ANNUITY', 'AMT_GOODS_PRICE',"TARGET"]]
amt = amt[(amt["AMT_GOODS_PRICE"].notnull()) & (amt["AMT_ANNUITY"].notnull())]
sns.pairplot(amt,hue="TARGET",palette=["b","r"])
plt.show()

In [ ]:
df.isnull().sum()

### LEBEL ENCODER AND ONE-HOT 
* Making Categorical variables Numeric

In [ ]:
df_cat = df.select_dtypes(include = [object])
del df_cat['data_type']

In [ ]:
df_cat.select_dtypes(include = [object]).apply(pd.Series.nunique, axis = 0)

In [ ]:
for col in list(df_cat.columns):
    df_cat[col].fillna(method = "ffill")

In [ ]:
df_cat.isnull().sum()

In [ ]:
print('df_cat shape:',df_cat.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in df_cat:
    if df_cat[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df_cat[col].unique())) <= 2:
            # Train on the training data
            le.fit(df_cat[col])
            # Transform both training and testing data
            df_cat[col] = le.transform(df_cat[col])
                        
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

In [ ]:
df_cat.head()

In [ ]:
df_cat = pd.get_dummies(df_cat)

In [ ]:
print('df_cat shape:',df_cat.shape)

In [ ]:
df_cat.head()

## Predictive Value Assignment (NaN) Methods
* Firstly we will look at outlier values then we predict Nan values

In [ ]:
df_num = df.select_dtypes(include = ['float64','int64'])
df_num.drop(['SK_ID_CURR','TARGET'],axis=1,inplace=True)

In [ ]:
IQR = df_num.describe().T
IQR['lower'] = IQR['25%']-1.5*(IQR['75%']-IQR['25%'])
IQR['upper'] = IQR['75%']+1.5*(IQR['75%']-IQR['25%'])
IQR.T

In [ ]:
IQR.T.iloc[:,9:]

In [ ]:
IQR.T.iloc[:,18:]

In [ ]:
IQR.T.iloc[:,27:]

* We are looking at some outlier of important variables 

In [ ]:
len(df[df['OBS_30_CNT_SOCIAL_CIRCLE']>5.000000])

In [ ]:
len(df[df['TOTALAREA_MODE']>0.259500])

In [ ]:
len(df[df['OWN_CAR_AGE']>30.000000])

In [ ]:
len(df[df['AMT_GOODS_PRICE']>1.336500e+06])  ####################There are many outlier 

In [ ]:
len(df[df['AMT_CREDIT']> 1.588894e+06])      

In [ ]:
len(df[df['AMT_ANNUITY']>62304.750000])

* There are lots of outlier in variable AMT_GOODS_PRICE.Thus we decided to covert nan values.

In [ ]:
df.loc[df['AMT_GOODS_PRICE'] > 1.336500e+06,'AMT_GOODS_PRICE']=np.nan

In [ ]:
df['AMT_GOODS_PRICE'].isnull().sum()

### Lets predict Nan values from machine learning method.

In [ ]:
df_num = df.select_dtypes(include = ['float64','int64'])
df_num.drop(['SK_ID_CURR','TARGET'],axis=1,inplace=True)

In [ ]:
df_num.isnull().sum()

In [ ]:
!pip install ycimpute

In [ ]:
from ycimpute.imputer import EM

In [ ]:
var_names = list(df_num) 

In [ ]:
var_names

In [ ]:
np_df_num =np.array(df_num)

In [ ]:
dff = EM().complete(np_df_num)

In [ ]:
dff = pd.DataFrame(dff, columns = var_names)

In [ ]:
dff.isnull().sum()

In [ ]:
dff.sample(10)

In [ ]:
dff_corr = dff.corr()
plt.figure(figsize = (28, 26))

# Heatmap of correlations
sns.heatmap(dff_corr, cmap = plt.cm.RdYlBu_r, vmin = -0.25,fmt='.0g', annot = True, vmax = 0.6)
plt.title('Correlation Heatmap')

# OUTLIER DATA ANALYSIS
### Confidence Distance Matrix Method

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
# n_neighbors = 10 komşuluk sayısı, contamination = 0.1 saydamlık
clf = LocalOutlierFactor(n_neighbors = 10, contamination = 0.1)

In [ ]:
clf.fit_predict(dff)

In [ ]:
# negatif skorlar 
dff_scores = clf.negative_outlier_factor_

In [ ]:
np.sort(dff_scores)[0:1000]

* Gözlemler için uzaklıklar matrisi kullanılarak skorlar elde edilmiştir.

In [ ]:
esik_deger = np.sort(dff_scores)[7]
esik_deger

In [ ]:
len(dff[dff_scores<esik_deger])

* -14.50 eşik değerinden daha küçük skora sahip gözlemlere baskılama yöntemi uygulanacaktır.

In [ ]:
dff[dff_scores==esik_deger]

In [ ]:
# eşik skora sahip gözlem baskılama verisi olarak belirleniyor
baskılama_deg = dff[dff_scores==esik_deger]

In [ ]:
# esik skordan daha küçük skora sahip gözlemler için True-False şeklinde ARRAY oluşturulyor
outlier_array = dff_scores<esik_deger
outlier_array

In [ ]:
# outlier_array'ın döndürdüğü True-False değerler ile filtreleme yapılarak Outlier gözlemler ile DATAFRAME oluşturuluyor
outlier_df = dff[outlier_array]
len(outlier_df)

In [ ]:
outlier_df 

In [ ]:
# outlier_df indexlerinden arındırılarak ARRAY'a dönüştürülüyor.
outlier_df.to_records(index=False)

In [ ]:
# Bu array res olarak tutuluyor.
res = outlier_df.to_records(index=False)

In [ ]:
# res'deki tüm veriler yerine baskılama dergerleri atanıyor
res[:] = baskılama_deg.to_records(index=False)

In [ ]:
res

In [ ]:
dff[outlier_array]

In [ ]:
# Bir array olan res aykırı gözlemlerin indexleri kullanılarak DATAFRAME dönüştürülüyor ve dff deki aykırı gözlemlerin yerine atanyor
dff[outlier_array] = pd.DataFrame(res, index = dff[outlier_array].index)

In [ ]:
dff[outlier_array]

* Uzaklıklar matrisi kullanılarak gözlemlenen 7 aykırı gözlem baskılama yöntemi ile manüple edilmiştir

In [ ]:
df_app = df[['data_type','SK_ID_CURR','TARGET']]

In [ ]:
print(df_app.shape,dff.shape,df_cat.shape)

In [ ]:
list(dff.columns)

In [ ]:
list(df_cat.columns)

In [ ]:
display(dff.sample(10))
display(df_cat.sample(10))
display(df.head())

In [ ]:
dff.insert(0,'data_type',df['data_type'].values)#1.kolona ekledik
dff.insert(1,'SK_ID_CURR',df['SK_ID_CURR'].values)  #2.kolona ekledik
dff.insert(2,'TARGET',df['TARGET'].values)      #3.kolona ekledik
df_cat.insert(0,'SK_ID_CURR',df['SK_ID_CURR'].values)   #kategorik dataya id ekledik merge yapmak icin

In [ ]:
df_cat.head()

In [ ]:
df1 = pd.merge( dff, df_cat, on='SK_ID_CURR')  #kategorik ve numerik ler birlestirildi

In [ ]:
df1.head()

In [ ]:
df1.isnull().sum()

In [ ]:
df1.to_csv (r'C:\Users\Dell\Desktop\proje\traintest.csv', index = False, header=True)

df1.head()

In [ ]:
#df1.to_csv(r'C:\Users\Dell\Desktop\proje\traintest.csv', index = False)